In [38]:
import numpy as np
import requests
import csv

In [27]:
def get_embedding(s: str):
    res = requests.post("http://localhost:1234/v1/embeddings", json={
        "input": s,
        "encoding_format": "float"
    })
    return np.array(res.json()["data"][0]["embedding"])

In [15]:
def cosine_similarity(a, b):
    """
    Calculates the cosine similarity between two NumPy vectors.
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    if norm_a == 0 or norm_b == 0:
        return 0 # Return 0 if either vector is a zero vector
    
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [40]:
restaurants = []
with open('test_data.csv', newline='') as csvfile:
    rows = csv.reader(csvfile, delimiter=',', quotechar='"')
    restaurants = [
        (title, description, get_embedding(description))
        for title, description in rows
    ][1:]

In [ ]:
def execute_query(query):
    query = get_embedding("hot food")
    x = sorted([ (cosine_similarity(restaurant[2], query), i) for i, restaurant in enumerate(restaurants)])
    return [ (restaurants[i][0], similarity) for (similarity, i) in x[-10:]]

In [46]:
execute_query("hot food")


[('Marugame Udon', np.float64(0.3908816345615723)),
 ('O2 Valley', np.float64(0.40404261546466236)),
 ('Poki Bowl', np.float64(0.4075935655382623)),
 ('Panda Express', np.float64(0.4303089433054471)),
 ('Pho 24', np.float64(0.44467056756098167)),
 ('Spartan Taco Truck', np.float64(0.44756859424828127)),
 ('Spoonfish Poke', np.float64(0.4486383198984743)),
 ('Super Taqueria', np.float64(0.45450871624466543)),
 ('Buffalo Wild Wings', np.float64(0.4603057622200309)),
 ('Orenchi Ramen', np.float64(0.46569522398697183))]